# classify using a simple classifier

### prepare the train and test

In [1]:
from PIL import Image
import numpy as np
import pandas as pd
import time
import matplotlib.pyplot as plt

import os
import pandas as pd

# Import the libraries
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.models import Model
from pathlib import Path

mycwd = os.path.dirname(os.getcwd())
# flags
language = "english"
#language = "persian"
#language = "italian"
search_engine = "qwant"
search_engines = ["bing", "qwant", "ecosia"]
#search_engines = ["qwant", "ecosia"]

# folders
folder_images = os.path.join(mycwd, "0-dataset", "test_images")
root_txt =  os.path.join(mycwd, "0-dataset", "test.data.v1.1.gold")
#folder_search = os.path.join(mycwd, "9-Insaf_scrapSearch", "scrape_"+language, search_engine, "query")

if language == "english":
    file_gold = os.path.join(root_txt,"en.test.gold.v1.1.txt")
    file_data = os.path.join(root_txt,"en.test.data.v1.1.txt")
elif language == "persian":
    file_gold = os.path.join(root_txt,"fa.test.gold.txt")
    file_data = os.path.join(root_txt,"fa.test.data.txt")
elif language == "italian":
    file_gold = os.path.join(root_txt,"it.test.gold.v1.1.txt")
    file_data = os.path.join(root_txt,"it.test.data.v1.1.txt") 

folder_features_all = os.path.join(mycwd, "12-Insaf_compute_features", "features_all")
file_features = os.path.join(folder_features_all, "features.csv")
file_names = os.path.join(folder_features_all, "image_names.csv")

folder_scrapped_features = os.path.join(mycwd, "12-Insaf_compute_features", "features")
    

df_data = pd.read_csv(file_data, delimiter='\t', header=None)
df_data.head()

df_gold = pd.read_csv(file_gold, delimiter='\t', header=None)
df_gold.head()

df_image_features = pd.read_csv(file_features, delimiter=',')
df_image_features.head()

df_image_names = pd.read_csv(file_names, delimiter='\t')
df_image_names.head()

,0
0,image.0.jpg
1,image.1.jpg
2,image.10.jpg
3,image.100.jpg
4,image.1000.jpg


In [6]:
missing_contexts = []
for i in range (df_gold.shape[0]):
    feature_path = os.path.join(folder_scrapped_features, language, search_engine+"_"+str(i)+".csv")    
    if os.path.isfile(feature_path)== False:
        missing_contexts.append(i)

In [7]:
missing_contexts

[19, 21, 23, 38, 40, 45, 46, 292, 302]

I have downloaded those images (from scrapping code), and computed their features (from feature computation code), now the classification phase

### Prepare the data and labels

In [100]:
#all_feat = np.zeros((d))
df_features = pd.DataFrame()
features_i = []
for i in range (df_gold.shape[0]):
    image_index = df_image_names[df_image_names["0"]==df_data.iloc[i][2]].index.values
    featureC = df_image_features.iloc[image_index]
    df_features = df_features.append(featureC)
    file_scrapped_features = os.path.join(folder_scrapped_features, language, search_engine +
                                          "_" + str(i) + ".csv") 
    featureDB = pd.read_csv(file_scrapped_features, delimiter=',')
    df_features = df_features.append(featureDB)
    

In [102]:
df_features

,0,1,2,3,4,5,6,7,8,9,...,4086,4087,4088,4089,4090,4091,4092,4093,4094,4095
3800,0.000000,0.036849,0.000000,0.000000,0.000000,0.000000,0.000000,0.071793,0.000000,0.033347,...,0.062409,0.000000,0.002975,0.000000,0.026942,0.000000,0.000000,0.000000,0.000000,0.000000
0,0.000000,0.000000,0.000000,0.027117,0.000000,0.000000,0.000000,0.017534,0.000000,0.002820,...,0.000000,0.000000,0.000000,0.009144,0.007342,0.033182,0.000000,0.000000,0.000000,0.000000
1,0.004738,0.000000,0.000000,0.000000,0.000000,0.022488,0.000000,0.033879,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.005606,0.000000,0.001500,0.000000,0.000000,0.029360,0.000000
2,0.002935,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.016983,0.000000,0.000000,0.000000,0.002928,0.000000,0.000000,0.000000,0.014986,0.011287
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.020835,0.000000,0.018372,0.000000,0.010210,...,0.000000,0.000000,0.000000,0.031686,0.000000,0.000000,0.000000,0.000000,0.001867,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45,0.000000,0.000000,0.000000,0.000000,0.000000,0.008387,0.000000,0.031690,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.030686,0.000000,0.004898,0.013936,0.000000,0.008109,0.000000
46,0.000000,0.022849,0.033434,0.039055,0.000000,0.000000,0.015561,0.000000,0.000000,0.000000,...,0.009730,0.058644,0.000000,0.000000,0.000000,0.000000,0.037968,0.000000,0.000000,0.000000
47,0.004506,0.000000,0.000000,0.074131,0.000000,0.010620,0.000000,0.000000,0.000000,0.000000,...,0.010077,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.011789,0.000000,0.000000
48,0.000000,0.045931,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.040015,0.029034,...,0.000000,0.035883,0.033544,0.000000,0.008194,0.000000,0.000000,0.000000,0.000000,0.041933


In [105]:
#all_feat = np.zeros((d))
df_labels = pd.DataFrame()
for i in range (df_gold.shape[0]):
    file_scrapped_features = os.path.join(folder_scrapped_features, language, search_engine +
                                          "_" + str(i) + ".csv") 
    featureDB = pd.read_csv(file_scrapped_features, delimiter=',')
    df_labels = df_labels.append(pd.DataFrame(np.zeros((featureDB.shape[0]+1))+i))

In [107]:
df_labels.shape

(23606, 1)

In [108]:
df_features.shape

(23606, 4096)

In [133]:
X= df_features.to_numpy(copy=True)
y = df_labels.to_numpy(copy=True)
y = y[:,0]

In [110]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB

In [136]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=0)
gnb = GaussianNB()
y_pred = gnb.fit(X_train, y_train).predict(X_test)
print("Number of mislabeled points out of a total %d points : %d"
% (X_test.shape[0], (y_test != y_pred).sum()))


Number of mislabeled points out of a total 2361 points : 1927


In [113]:
X_train.shape

(11803, 4096)

In [132]:
y_test[:,0].shape

(11803,)

In [129]:
np.transpose(y_test).shape

(1, 11803)

In [117]:
(X_test.shape[0], (y_test != y_pred).sum())

(11803, 139049056)

In [123]:
y_pred[0].shape

()

In [138]:
from sklearn.linear_model import SGDClassifier
clf = SGDClassifier(loss="hinge", penalty="l2", max_iter=10)
clf.fit(X, y)

C:\Users\Insaf\anaconda3\envs\competition01\lib\site-packages\sklearn\linear_model\least_angle.py:35: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  eps=np.finfo(np.float).eps,
C:\Users\Insaf\anaconda3\envs\competition01\lib\site-packages\sklearn\linear_model\least_angle.py:597: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  eps=np.finfo(np

SGDClassifier(alpha=0.0001, average=False, class_weight=None,
       early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
       l1_ratio=0.15, learning_rate='optimal', loss='hinge', max_iter=5,
       n_iter=None, n_iter_no_change=5, n_jobs=None, penalty='l2',
       power_t=0.5, random_state=None, shuffle=True, tol=None,
       validation_fraction=0.1, verbose=0, warm_start=False)

In [ ]:
clf = SGDClassifier(loss="hinge", penalty="l2", max_iter=10)
clf.fit(X, y)

In [145]:
y_pred = clf.predict(X_test)

In [146]:
print("Number of mislabeled points out of a total %d points : %d"
% (X_test.shape[0], (y_test != y_pred).sum()))

Number of mislabeled points out of a total 2361 points : 342


In [143]:
y_pred[1]

227.0

### prepare the data of the contexts

In [184]:
clf = SGDClassifier(loss="modified_huber", max_iter=30).fit(X, y)

C:\Users\Insaf\anaconda3\envs\competition01\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:183: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If max_iter is set but tol is left unset, the default value for tol in 0.19 and 0.20 will be None (which is equivalent to -infinity, so it has no effect) but will change in 0.21 to 1e-3. Specify tol to silence this warning.
  FutureWarning)
C:\Users\Insaf\anaconda3\envs\competition01\lib\site-packages\sklearn\utils\fixes.py:357: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if _joblib.__version__ >= LooseVersion('0.12'):


In [185]:
#all_feat = np.zeros((d))
matches = np.zeros((df_data.shape[0], 10))
for i in range (df_gold.shape[0]):
    for j in range (2,df_data.shape[1]):
        image_index = df_image_names[df_image_names["0"]==df_data.iloc[i][j]].index.values
        featureC = df_image_features.iloc[image_index]
        matches[i,j-2]= clf.predict_proba(featureC.to_numpy(copy=True).reshape(1,-1))[0][i]
        """print("treating image ", j)
        print(clf.predict_proba(featureC.to_numpy(copy=True).reshape(1,-1))[0][i])
    """

In [186]:
matches_df = pd.DataFrame(matches)

In [187]:
matches_df

,0,1,2,3,4,5,6,7,8,9
0,0.320607,0.000000,0.000000,0.000000,0.000000,0.138290,0.000000,0.000000,0.344579,0.000000
1,0.510752,0.000000,0.000000,0.000000,0.000000,0.007056,0.000000,0.000000,0.000000,0.000000
2,0.491299,0.000000,0.000000,0.000000,0.000000,0.012264,0.000000,0.000000,0.000000,0.000000
3,0.340744,0.000000,0.000000,0.000000,0.000000,0.000000,0.143884,0.010515,0.000000,0.000000
4,0.576725,0.000000,0.025201,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...
458,0.264642,0.000000,0.021139,0.012612,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
459,0.160732,0.036255,0.000000,0.031706,0.029881,0.046240,0.045726,0.000000,0.133713,0.080122
460,0.320252,0.000000,0.000000,0.645974,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
461,0.378705,0.000000,0.000000,0.158792,0.000000,0.000000,0.180017,0.035107,0.000000,0.180447


In [191]:
final_result_images = []
final_result_distances = []
for idx, row in matches_df.iterrows():
    dist_test = row
    ranked_dist = -np.sort(-dist_test)
    ranked_dist_indices = np.argsort(-dist_test)
    sorted_df = np.array(df_data.iloc[idx][ranked_dist_indices+2])
    final_result_images.append(sorted_df)
    final_result_distances.append(ranked_dist)

In [192]:
result_tocsv = pd.DataFrame()
result_tocsv["language"]=[]
result_tocsv["data"]=[]
result_tocsv["candidate"]=final_result_images
result_tocsv["relevance"]=final_result_distances
result_tocsv["language"]=language
result_tocsv["gold"]=df_gold.values
result_tocsv["data"]=range(len(final_result_distances))

In [193]:
result_tocsv

,language,data,candidate,relevance,gold
0,english,0,"[image.2166.jpg, image.4418.jpg, image.4415.jp...","[0.34457918364900364, 0.32060663573968956, 0.1...",image.2166.jpg
1,english,1,"[image.4429.png, image.4427.jpg, image.4422.jp...","[0.5107523260388563, 0.007055607515222442, 0.0...",image.4429.png
2,english,2,"[image.4435.jpg, image.4432.jpg, image.4436.jp...","[0.4912994446435285, 0.012263879203628645, 0.0...",image.4432.jpg
3,english,3,"[image.4439.jpg, image.1435.jpg, image.4446.pn...","[0.3407435548358683, 0.14388418060641275, 0.01...",image.1435.jpg
4,english,4,"[image.4454.jpg, image.4455.jpg, image.4450.jp...","[0.5767245507579232, 0.0252005534189324, 0.0, ...",image.4455.jpg
...,...,...,...,...,...
458,english,458,"[image.8063.jpg, image.4891.jpg, image.7450.jp...","[0.2646424995544188, 0.021138835508585776, 0.0...",image.8063.jpg
459,english,459,"[image.8073.jpg, image.8067.jpg, image.8072.jp...","[0.16073182880838535, 0.13371317099079946, 0.0...",image.8069.jpg
460,english,460,"[image.8081.jpg, image.8082.jpg, image.8079.jp...","[0.6459736694721107, 0.32025237626459924, 0.0,...",image.8081.jpg
461,english,461,"[image.8087.jpg, image.8089.jpg, image.8088.jp...","[0.3787054468898943, 0.1804472147232383, 0.180...",image.8089.jpg


In [196]:
# save the csv of results
result_path = "Results"
if not os.path.exists(result_path):
    os.makedirs(result_path)   
result_name =os.path.join(result_path, "test_sorting4.csv" )
result_tocsv.to_csv(result_name)

In [197]:
result_path

'Results'

# Test another classifier

In [200]:
from sklearn import tree
clf2 = tree.DecisionTreeClassifier()
clf2 = clf2.fit(X, y)

C:\Users\Insaf\anaconda3\envs\competition01\lib\site-packages\sklearn\tree\tree.py:149: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  y_encoded = np.zeros(y.shape, dtype=np.int)


In [201]:
#all_feat = np.zeros((d))
matches = np.zeros((df_data.shape[0], 10))
for i in range (df_gold.shape[0]):
    for j in range (2,df_data.shape[1]):
        image_index = df_image_names[df_image_names["0"]==df_data.iloc[i][j]].index.values
        featureC = df_image_features.iloc[image_index]
        matches[i,j-2]= clf2.predict_proba(featureC.to_numpy(copy=True).reshape(1,-1))[0][i]
        
matches_df = pd.DataFrame(matches)
final_result_images = []
final_result_distances = []
for idx, row in matches_df.iterrows():
    dist_test = row
    ranked_dist = -np.sort(-dist_test)
    ranked_dist_indices = np.argsort(-dist_test)
    sorted_df = np.array(df_data.iloc[idx][ranked_dist_indices+2])
    final_result_images.append(sorted_df)
    final_result_distances.append(ranked_dist)
    
result_tocsv = pd.DataFrame()
result_tocsv["language"]=[]
result_tocsv["data"]=[]
result_tocsv["candidate"]=final_result_images
result_tocsv["relevance"]=final_result_distances
result_tocsv["language"]=language
result_tocsv["gold"]=df_gold.values
result_tocsv["data"]=range(len(final_result_distances))

# save the csv of results
result_path = "Results"
if not os.path.exists(result_path):
    os.makedirs(result_path)   
    
result_name =os.path.join(result_path, "test_sorting5.csv" )
result_tocsv.to_csv(result_name)

# End of test another classifier

# Test MLP

In [204]:
from sklearn.neural_network import MLPClassifier
#clf3 = MLPClassifier(solver='lbfgs', alpha=1e-5,hidden_layer_sizes=(5, 2), random_state=1)
clf3 = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(15,), random_state=1, max_iter=2000)
clf3.fit(X, y)

MLPClassifier(activation='relu', alpha=1e-05, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(15,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=2000, momentum=0.9,
       n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
       random_state=1, shuffle=True, solver='lbfgs', tol=0.0001,
       validation_fraction=0.1, verbose=False, warm_start=False)

In [207]:
#all_feat = np.zeros((d))
matches = np.zeros((df_data.shape[0], 10))
for i in range (df_gold.shape[0]):
    for j in range (2,df_data.shape[1]):
        image_index = df_image_names[df_image_names["0"]==df_data.iloc[i][j]].index.values
        featureC = df_image_features.iloc[image_index]
        matches[i,j-2]= clf3.predict_proba(featureC.to_numpy(copy=True).reshape(1,-1))[0][i]
        
matches_df = pd.DataFrame(matches)
final_result_images = []
final_result_distances = []
for idx, row in matches_df.iterrows():
    dist_test = row
    ranked_dist = -np.sort(-dist_test)
    ranked_dist_indices = np.argsort(-dist_test)
    sorted_df = np.array(df_data.iloc[idx][ranked_dist_indices+2])
    final_result_images.append(sorted_df)
    final_result_distances.append(ranked_dist)
    
result_tocsv = pd.DataFrame()
result_tocsv["language"]=[]
result_tocsv["data"]=[]
result_tocsv["candidate"]=final_result_images
result_tocsv["relevance"]=final_result_distances
result_tocsv["language"]=language
result_tocsv["gold"]=df_gold.values
result_tocsv["data"]=range(len(final_result_distances))

# save the csv of results
result_path = "Results"
if not os.path.exists(result_path):
    os.makedirs(result_path)   
    
result_name =os.path.join(result_path, "test_sorting7.csv" )
result_tocsv.to_csv(result_name)

# end of test MLP

# try logistic regression

In [212]:
from sklearn.linear_model import LogisticRegression

start_time = time.time()

lr_clf = LogisticRegression(max_iter=6000)
lr_clf.fit(X, y)

print(round(time.time() - start_time), "seconds")

C:\Users\Insaf\anaconda3\envs\competition01\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Insaf\anaconda3\envs\competition01\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


781 seconds


In [213]:
#all_feat = np.zeros((d))
matches = np.zeros((df_data.shape[0], 10))
for i in range (df_gold.shape[0]):
    for j in range (2,df_data.shape[1]):
        image_index = df_image_names[df_image_names["0"]==df_data.iloc[i][j]].index.values
        featureC = df_image_features.iloc[image_index]
        matches[i,j-2]= lr_clf.predict_proba(featureC.to_numpy(copy=True).reshape(1,-1))[0][i]
        
matches_df = pd.DataFrame(matches)
final_result_images = []
final_result_distances = []
for idx, row in matches_df.iterrows():
    dist_test = row
    ranked_dist = -np.sort(-dist_test)
    ranked_dist_indices = np.argsort(-dist_test)
    sorted_df = np.array(df_data.iloc[idx][ranked_dist_indices+2])
    final_result_images.append(sorted_df)
    final_result_distances.append(ranked_dist)
    
result_tocsv = pd.DataFrame()
result_tocsv["language"]=[]
result_tocsv["data"]=[]
result_tocsv["candidate"]=final_result_images
result_tocsv["relevance"]=final_result_distances
result_tocsv["language"]=language
result_tocsv["gold"]=df_gold.values
result_tocsv["data"]=range(len(final_result_distances))

# save the csv of results
result_path = "Results"
if not os.path.exists(result_path):
    os.makedirs(result_path)   
    
result_name =os.path.join(result_path, "test_sorting9.csv" )
result_tocsv.to_csv(result_name)

# End of Logistic regression

# try to train only on the 10 contexts

In [ ]:
#all_feat = np.zeros((d))
df_features = pd.DataFrame()
df_labels = pd.DataFrame()
for i in range (df_gold.shape[0]):
    for j in range(2, df_data.shape[1]):
        image_index = df_image_names[df_image_names["0"]==df_data.iloc[i][j]].index.values
        featureC = df_image_features.iloc[image_index]
        df_features = df_features.append(featureC)
        file_scrapped_features = os.path.join(folder_scrapped_features, language, search_engine +
                                          "_" + str(image_index) + ".csv") 
        featureDB = pd.read_csv(file_scrapped_features, delimiter=',')
        df_features = df_features.append(featureDB)
        
        label_index = df_gold[df_gold==df_data.iloc[i][j]].index
        df_labels = df_labels.append(pd.DataFrame(np.zeros((featureDB.shape[0]+1))+label_index))
    break
        
        
    

In [ ]:
#all_feat = np.zeros((d))
df_labels = pd.DataFrame()
for i in range (df_gold.shape[0]):
    file_scrapped_features = os.path.join(folder_scrapped_features, language, search_engine +
                                          "_" + str(i) + ".csv") 
    featureDB = pd.read_csv(file_scrapped_features, delimiter=',')
    df_labels = df_labels.append(pd.DataFrame(np.zeros((featureDB.shape[0]+1))+i))

# try to train only on the 10 contexts

In [151]:
y_pred = clf.predict(X_test)
print("Number of mislabeled points out of a total %d points : %d"
% (X_test.shape[0], (y_test != y_pred).sum()))

Number of mislabeled points out of a total 2361 points : 142


In [154]:
len(X_test[0])

4096

In [164]:
predictions = clf.predict_proba(X_test[0].reshape(1,-1))

In [165]:
predictions

array([[0.65068723, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.17487587,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.  

In [160]:
predictions[0][0]

0.6506872342006732